# Alternating Least Squares Implicit Collaborative Filtering

- https://jessesw.com/Rec-System/

## Processing the Data

In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve

In [2]:
# df_action = pd.read_csv('../input/제6회 L.POINT Big Data Competition-분석용데이터-01.온라인 행동 정보.csv', parse_dates=['sess_dt'])
df_transaction = pd.read_csv('../input/제6회 L.POINT Big Data Competition-분석용데이터-02.거래 정보.csv', parse_dates=['de_dt'])
# df_client_demo = pd.read_csv('../input/제6회 L.POINT Big Data Competition-분석용데이터-03.고객 Demographic 정보.csv')
df_product = pd.read_csv('../input/제6회 L.POINT Big Data Competition-분석용데이터-04.상품분류 정보.csv')

In [3]:
# convert df_product['pd_c'] datatype : int -> object
df_product['pd_c'] = df_product['pd_c'].apply(lambda num: "{:04n}".format(num))

# df_transaction and df_prodcct merge!
df_transaction = pd.merge(df_transaction, df_product, how='left')
print(df_transaction.shape)

(599961, 12)


In [4]:
# pd_c가 unknown은 drop
df_transaction.dropna(axis=0, inplace=True)
print(df_transaction.shape)
df_transaction.head()

(582283, 12)


,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct,clac_nm1,clac_nm2,clac_nm3
4,39423,105124,13,A03,0565,2019-09-20,17:26,2990,1,Fruits,Imported Fruits,Bananas
5,21279,104907,1,A03,0565,2019-09-20,10:27,2990,1,Fruits,Imported Fruits,Bananas
6,48969,104882,5,A03,0572,2019-09-20,10:05,7490,1,Fruits,Imported Fruits,Kiwi
7,30533,103245,5,A03,0670,2019-09-20,14:36,49800,1,Grains,Rice,Rice
8,64346,104317,1,A03,0543,2019-09-20,22:34,26900,1,Fruits,Domestic Fruits,Pears


In [5]:
df_product.head()

,pd_c,clac_nm1,clac_nm2,clac_nm3
0,0001,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Automobile Oil / Additives
1,0002,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Car Lights
2,0003,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Car Paint
3,0004,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Filters
4,0005,Automotive Products,Automotive Replacement Repair / Maintanance Kits,Wiper Blades


In [6]:
df = df_transaction[['clnt_id', 'pd_c', 'buy_ct']]
grouped_df =df.groupby(['clnt_id', 'pd_c']).sum().reset_index()

# Replace a sum of zero purchases with a one to indicate purchased
grouped_df.buy_ct.loc[grouped_df.buy_ct == 0] = 1 

print(grouped_df.shape)
grouped_df.head(10)

(327638, 3)


,clnt_id,pd_c,buy_ct
0,2,0092,26
1,2,0151,1
2,2,0186,1
3,2,0189,1
4,2,0351,1
5,2,0857,1
6,2,1015,1
7,2,1395,4
8,9,0114,1
9,9,0188,1


### Instead of representing an explicit rating, the purchase quantity can represent a "confidence" in terms of how strong the interaction was.

In [7]:
# Only get customers where purchase totals were positive
grouped_purchased = grouped_df.query('buy_ct > 0')
print(grouped_purchased.shape)
grouped_purchased.head()

(327638, 3)


,clnt_id,pd_c,buy_ct
0,2,0092,26
1,2,0151,1
2,2,0186,1
3,2,0189,1
4,2,0351,1


In [8]:
customers = list(np.sort(grouped_purchased.clnt_id.unique()))
products = list(grouped_purchased.pd_c.unique())
quantity = list(grouped_purchased.buy_ct)
print(len(customers), len(products), len(quantity))

11270 1665 327638


In [9]:
rows = grouped_purchased.clnt_id.astype('category', categoried = customers).cat.codes.astype(int)
cols = grouped_purchased.pd_c.astype('category', categoried = products).cat.codes.astype(int)

# Create a sparse matrix for our users and mcats containing number of purchases
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

In [10]:
purchases_sparse

<11270x1665 sparse matrix of type '<class 'numpy.int64'>'
	with 327638 stored elements in Compressed Sparse Row format>

In [11]:
# 98.3% of the interaction matrix is sparse.
matrix_size = purchases_sparse.shape[0] * purchases_sparse.shape[1]
# Number of items interacted with
num_purchases = len(purchases_sparse.nonzero()[0])
sparsity = 100 * (1 - (num_purchases/matrix_size))
sparsity

98.25395226637463

### Creating a Traning and Validation Set

In [12]:
import random

This will return our training set, a test set that has been binarized to 0/1 for purchased/not purchased, and a list of which users had at least one item masked. We will test the performance of the recommender system on these users only. I am masking 20% of the user/item interactions for this example.

In [13]:
def make_train(ratings, pct_test=0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    # Make a copy of the original set to be the test set. 
    test_set = ratings.copy()
    # Store the test set as a binary preference matrix
    test_set[test_set != 0] = 1
    # Make a copy of the original data we can alter as our training set.
    training_set = ratings.copy()
    # Find the indices in the ratings data where an interaction exists
    nonzero_inds = training_set.nonzero() 
    # Zip these pairs together of user,item index into list
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1]))
    # Set the random seed to zero for reproducibility
    random.seed(0)
    # Round the number of samples needed to the nearest integer
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs)))
    # Sample a random number of user-item pairs without replacement
    samples = random.sample(nonzero_pairs, num_samples)
    # Get the user row indices
    user_inds = [index[0] for index in samples]
    # Get the item column indices
    item_inds = [index[1] for index in samples]
    # Assign all of the randomly chosen user-item pairs to zero
    training_set[user_inds, item_inds] = 0 
    # Get rid of zeros in sparse array storage after update to save space
    training_set.eliminate_zeros()
    
    # Output the unique list of user rows that were altered 
    return training_set, test_set, list(set(user_inds))

In [14]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test=0.2)

In [15]:
product_train.shape, product_test.shape

((11270, 1665), (11270, 1665))

In [16]:
def implicit_weighted_ALS(training_set, lambda_val=0.1, alpha=40, iterations=10, rank_size=20, seed=0):
    '''
    Implicit weighted ALS taken from Hu, Koren, and Volinsky 2008. Designed for alternating least squares and implicit
    feedback based collaborative filtering. 
    
    parameters:
    
    training_set - Our matrix of ratings with shape m x n, where m is the number of users and n is the number of items.
    Should be a sparse csr matrix to save space. 
    
    lambda_val - Used for regularization during alternating least squares. Increasing this value may increase bias
    but decrease variance. Default is 0.1. 
    
    alpha - The parameter associated with the confidence matrix discussed in the paper, where Cui = 1 + alpha*Rui. 
    The paper found a default of 40 most effective. Decreasing this will decrease the variability in confidence between
    various ratings.
    
    iterations - The number of times to alternate between both user feature vector and item feature vector in
    alternating least squares. More iterations will allow better convergence at the cost of increased computation. 
    The authors found 10 iterations was sufficient, but more may be required to converge. 
    
    rank_size - The number of latent features in the user/item feature vectors. The paper recommends varying this 
    between 20-200. Increasing the number of features may overfit but could reduce bias. 
    
    seed - Set the seed for reproducible results
    
    returns:
    
    The feature vectors for users and items. The dot product of these feature vectors should give you the expected 
    "rating" at each point in your original matrix. 
    '''
    
    # first set up our confidence matrix
    
    # To allow the matrix to stay sparse, I will add one later when each row is taken and converted to dence
    conf = (alpha*training_set) 
    num_user = conf.shape[0]
    # Get the size of our original ratings matrix, m X n 
    num_item = conf.shape[1]
    
    # initialize our X/Y feature vectors randomly with a set seed
    rstate = np.random.RandomState(seed)
    
    # Random numbers in a m x rank shape
    X = sparse.csr_matrix(rstate.normal(size = (num_user, rank_size)))
    # Normally this would be rank x n but we can transpose at the end. Makes calculation more simple.
    Y = sparse.csr_matrix(rstate.normal(size = (num_item, rank_size)))
    
    X_eye = sparse.eye(num_user)
    Y_eye = sparse.eye(num_item)
    # Our regularization term lambda * I
    lambda_eye = lambda_val * sparse.eye(rank_size)
    
    # We can compute this before iteration starts.
    
    # Begin iterations
    
    # Iterate back and forth between solving X given fixed Y and vice verse
    for iter_step in range(iterations): 
        # Compute yTy and xTx at begining of each iteration to save computing time
        yTy = Y.T.dot(Y)
        xTx = X.T.dot(X)
        
        # Being iteration to solve for X based on fixed Y 
        for u in range(num_user):
            # Grab user row from confidence matrix and convert to dence
            conf_samp = conf[u, :].toarray()
            pref = conf_samp.copy()
            pref[pref != 0] = 1  # Create binarized preference vector 
            CuI = sparse.diags(conf_samp, [0]) # Get Cu - I term, don't need to subtract 1 since we never added it 
            yTCuIY = Y.T.dot(CuI).dot(Y) # This is the yT(Cu-I)Y term
            yTCupu = Y.T.dot(CuI + Y_eye).dot(pref.T) # This is the yTCuPu term, where we add the eye back in Cu - I + I = Cu
            X[u] = spsolve(yTy + yTCuIY + lambda_eye, yTCupu)
            # Solve for Xu = ((yTy + yT(Cu-I)Y + lambda*I)^-1)yTCuPu, equation 4 from the paper
            
        # Begin iteration to solve for Y based on fixed X
        for i in range(num_item):
            conf_samp = conf[:, i].T.toarray() # Transpose to get it in row format and convert to dense
            pref = conf_samp.copy()
            pref[pref != 0] = 1 # Create binarized preference vector
            CiI = sparse.diags(conf_samp, [0]) # Get Ci-I term, don't need to subtract 1 since we never added it
            xTCiIX = X.T.dot(CiI).dot(X) # This is the xT(Cu-I)X term
            xTCiPi = X.T.dot(CiI + X_eye).dot(pref.T) # this is the xTCiPi term
            Y[i] = spsolve(xTx + xTCiIX + lambda_eye, xTCiPi)
            # Solve for Yi = ((xTx + xT(Cu-I)X + lambda*I)^-1)xTCiPi, equtation 5 from paper
            
        # End iterations
        # Transpose at the end to make up for not being transposed at the begining.
        # Y needs to be rank x n. Keep these as separate matrices for scale reasons 
        return X, Y.T  

In [17]:
user_vecs, item_vecs = implicit_weighted_ALS(product_train, lambda_val=0.1, alpha=15, iterations=10, rank_size=20)

In [18]:
user_vecs[0, :].dot(item_vecs).toarray()[0, :10]

array([-9.07579489e-06, -2.25386050e-05, -7.62959485e-05, -1.90207270e-05,
        3.38033606e-05, -7.90647854e-04, -1.20627512e-04, -6.01797842e-05,
       -1.42114880e-05, -5.07013351e-05])

### Evaluating the Recommender System

In [19]:
from sklearn import metrics

In [20]:
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

In [21]:
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    # An empty list to store the AUC for each user that had an item removed from the trainig set
    store_auc = [] 
    # To store popular AUC scores
    popularity_auc = []
    # Get sum of item iterations to find most popular 
    pop_items = np.array(test_set.sum(axis=0)).reshape(-1)
    item_vecs = predictions[1]
    # Iterate through each user that had an item altered 
    for user in altered_users:
        training_row = training_set[user,:].toarray().reshape(-1)# Get sum of item iteractions to find most popular
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs.T).toarray()[0, zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0, zero_inds].reshape(-1)
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
        
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))
    # Return the mean AUC rounded to there decimal places for both test and popularity benchmark
        

In [22]:
# AUC for our recommender system
calc_mean_auc(product_train, product_users_altered, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)

(0.8, 0.891)

Our recommender system had a mean AUC of 0.80, while the popular item benchmark had a higher AUC of 0.89. 

### A Recommendation Example
Let's examine the recommendations given to a particular user and decide subjectively if they make any sense.

we need to find a way of retrieving the items already purchased by a user in the training set. Initially, we will create an array of our customers and items we made earlier.

In [23]:
# Array of customer IDs from the ratings matrix
customers_arr = np.array(customers)
# Array of product IDs from the ratings matrix
products_arr = np.array(products)

In [77]:
def get_items_purchased(customer_id, df_transaction, df_product):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    customer_id - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    customers_list - The array of customers used in the ratings matrix
    
    products_list - The array of products used in the ratings matrix
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    returns:
    
    A list of item IDs and item descriptions for a particular customer that were already purchased in the training set
    '''
    # Returns the index row of our customer id
#     cust_ind = np.where(customers_list == customer_id)[0][0]
    
    # Get column indics of purchased items
#     purchased_ind = mf_train[cust_ind,:].nonzero()[1]
    
    # Get the stock codes for our purchased items
#     prod_codes = products_list[purchased_ind]
#     return item_lookup.loc[item_lookup.pd_c.isin(prod_codes)]
    purchase_items = df_transaction.loc[df_transaction['clnt_id'] == customer_id].pd_c.to_list()
    return df_product[df_product.pd_c.isin(purchase_items)]

In [79]:
customers_arr

array([    2,     9,    12, ..., 72410, 72423, 72424])

In [80]:
get_items_purchased(72410, df_transaction, df_product)

,pd_c,clac_nm1,clac_nm2,clac_nm3
97,0098,Beverages,Fruit and Vegetable Drinks,Mixed Fruit / Vegetable Drinks
113,0114,Beverages,Water,Water
185,0186,Chilled Foods,Fish Cakes and Crab Sticks,Crab Sticks
187,0188,Chilled Foods,Fish Cakes and Crab Sticks,Fish Cakes
188,0189,Chilled Foods,Fish Cakes and Crab Sticks,Fried Tofu
191,0192,Chilled Foods,Ham and Sausages,Ham
197,0198,Chilled Foods,Packaged Side Dishes,Pickled Radishes
235,0236,Coffee / Tea,Brewed Coffee,Tea Bags and Ground Coffee
335,0336,Cosmetics / Beauty Care,Skin Care,Essence / Serums
347,0348,Dairy Products,Milk,Functional Milk
